# Set_Up

In [122]:
strategy   = 'Kalman'
process    = 'Train'
symbol     = 'BTCUSD'
direction  = 'Short'
time_frame = 'M5'

root_data = f'/content/drive/MyDrive/Course Folder/Forex/XAUUSD/'
print(root_data)

rolling_window = 100

/content/drive/MyDrive/Course Folder/Forex/XAUUSD/


# Import_Libraries

In [123]:
!pip install ta-lib
import talib as ta
print(ta.__version__)

0.6.7


In [124]:
# Import libraries for data manipulation
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from datetime import timedelta

# For machine learning models
import xgboost
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split # Import train_test_split

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.models import load_model

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

# Import matplotlib as an alias plt and set the style
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn-v0_8-whitegrid")

# Import sys to append the path for custom function file
import sys
sys.path.append("..")

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

import joblib

In [125]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Support Functions

In [126]:
def results(data, pnl_column='PnL'):
    # Calculate the metrics
    time_difference = data.index.max() - data.index.min()
    days = time_difference.days
    total_trades = data[data['Open_Trade'].notna() & (data['Open_Trade'] != 0)].shape[0]
    profit_trades = data[data[pnl_column] > 0].shape[0]
    loss_trades = data[data[pnl_column] < 0].shape[0]
    profits = data[data[pnl_column] > 0][pnl_column].sum()
    losses = data[data[pnl_column] < 0][pnl_column].sum()

    # Create a dictionary with the results
    results_dict = {
        'days': days,
        'total_trades': total_trades,
        '': '',
        'income': profits,
        'losses': losses,
        'profits': profits + losses,
        ' ':' ',
        'profit_trades': profit_trades,
        'loss_trades': loss_trades,
        '  ':'  ',
        '% Win_Trades': profit_trades / (profit_trades + loss_trades) * 100 if (profit_trades + loss_trades) > 0 else 0,
        '% Loss_Trades': loss_trades / (profit_trades + loss_trades)*100 if (profit_trades + loss_trades) > 0 else 0
    }

    # Create a DataFrame from the dictionary and transpose it
    perf_metrics = pd.DataFrame([results_dict]).T

    # Rename the column
    perf_metrics.rename(columns={0: 'Results'}, inplace=True)

    # Format the DataFrame for display
    perf_metrics.loc[['days', 'total_trades', 'profit_trades', 'loss_trades','% Win_Trades','% Loss_Trades'], 'Results'] = perf_metrics.loc[['days', 'total_trades', 'profit_trades', 'loss_trades','% Win_Trades','% Loss_Trades'], 'Results'].apply(lambda x: f"{x:,.0f}" if pd.notna(x) else '')
    perf_metrics.loc[['income', 'losses', 'profits'], 'Results'] = perf_metrics.loc[['income', 'losses', 'profits'], 'Results'].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else '')

    return perf_metrics

In [127]:
def create_features(train_data, index):
    short_periods = [3, 5, 7, 10, 15, 17]
    long_periods = [20, 22, 66, 126, 252]
    periods = short_periods + long_periods

    features = pd.DataFrame(index=index)

    # Indicators that do not depend on the lookback period
    features['OBV'] = ta.OBV(train_data['Close'], train_data['Volume'])
    features['AD'] = ta.AD(train_data['High'], train_data['Low'],
                           train_data['Close'], train_data['Volume'])

    # Pre-compute moving averages to avoid repeated calculations
    sma = {p: ta.SMA(train_data['Close'], timeperiod=p) for p in periods}
    ema = {p: ta.EMA(train_data['Close'], timeperiod=p) for p in periods}

    for period in periods:
        features[f'RSI_{period}'] = ta.RSI(train_data['Close'], timeperiod=period)
        features[f'MFI_{period}'] = ta.MFI(train_data['High'], train_data['Low'],
                                           train_data['Close'], train_data['Volume'],
                                           timeperiod=period)
        features[f'ADX_{period}'] = ta.ADX(train_data['High'], train_data['Low'],
                                           train_data['Close'], timeperiod=period)
        features[f'ROCP_{period}'] = ta.ROCP(train_data['Close'], timeperiod=period)

    for s in short_periods:
        for l in long_periods:
            features[f'SMA_Crossover_{s}_{l}'] = sma[s] - sma[l]
            features[f'EMA_Crossover_{s}_{l}'] = ema[s] - ema[l]

    features.dropna(inplace=True)
    return features


In [128]:
def strategy_returns_dynamic_different_thresholds(prices, threshold):
    rolling = prices['Rolling Returns'].to_numpy()
    yearly_std = prices['Yearly Stdev'].to_numpy()
    pct_change = prices['Close'].pct_change().to_numpy()
    signals = np.zeros(len(prices), dtype=np.int8)

    curr_pos = 0
    hold_days = 0
    for i in range(len(prices)):
        if curr_pos == 0 or hold_days == 20:
            thresh_val = threshold * yearly_std[i]
            if rolling[i] >= thresh_val:
                curr_pos = 1
                hold_days = 0
            elif rolling[i] < -thresh_val:
                curr_pos = -1
                hold_days = 0
            else:
                curr_pos = 0
                hold_days = 0
        else:
            hold_days += 1
        signals[i] = curr_pos

    prices[f'Signal_{threshold}'] = signals
    strategy_returns = pct_change * np.roll(signals, 1)
    prices[f'Strategy Returns_{threshold}'] = strategy_returns
    return np.cumprod(strategy_returns + 1)


def strategy_returns_different_thresholds(prices, threshold):
    rolling = prices['Rolling Returns'].to_numpy()
    pct_change = prices['Close'].pct_change().to_numpy()
    signals = np.zeros(len(prices), dtype=np.int8)

    curr_pos = 0
    hold_days = 0
    for i in range(len(prices)):
        if curr_pos == 0 or hold_days == 20:
            if rolling[i] >= threshold:
                curr_pos = 1
                hold_days = 0
            elif rolling[i] < threshold:
                curr_pos = -1
                hold_days = 0
        else:
            hold_days += 1
        signals[i] = curr_pos

    prices[f'Signal_{threshold}'] = signals
    strategy_returns = pct_change * np.roll(signals, 1)
    prices[f'Strategy Returns_{threshold}'] = strategy_returns
    return np.cumprod(strategy_returns + 1)


# Data

In [129]:
data_type = 'Scale'

In [130]:
### Open OHLC dataframe

ohlc = pd.read_csv(root_data + 'Data/'+symbol+'_M5.csv', index_col=0)
ohlc.index = pd.to_datetime(ohlc.index)
time_difference = ohlc.index.max() - ohlc.index.min()
number_of_days = time_difference.days

print(f"The train_data DataFrame covers a period of {number_of_days} days.")
ohlc.tail(3)

The train_data DataFrame covers a period of 937 days.


,Open,High,Low,Close,Volume,Spread
Date,,,,,,
2025-07-26 23:45:00,118007.00,118100.13,117967.50,118023.88,757,100
2025-07-26 23:50:00,118023.63,118052.38,117951.88,118005.75,547,0
2025-07-26 23:55:00,118008.75,118049.38,118008.50,118042.63,485,587


In [141]:
### Features

features_5m = pd.read_csv(root_data + 'Results/'+symbol+'_'+direction+'_M5M10_'+data_type+'_Features.csv', index_col=0)
features_5m['Date'] = features_5m.index
features_5m['Date'] = pd.to_datetime(features_5m['Date'])
features_5m.set_index("Date", inplace=True)

print(features_5m.shape)
features_5m.tail(3)

(14438, 272)


,label,10min_RSI_3_diff,10min_RSI_3,10min_slope_signal_600_3,10min_slope_signal_300_3,10min_slope_div_300_3_diff,10min_slope_lin_reg_signal_900_6,RSI_7,10min_slope_signal_900_3,10min_slope_angle_900_3_diff,...,10min_slope_angle_600_9 - slope_angle_900_6,slope_signal_600_3 - slope_signal_600_6,slope_lin_reg_signal_600_9,slope_angle_300_3 - slope_angle_900_6,slope_angle_900_6,10min_slope_angle_300_9 - slope_angle_600_6,10min_slope_angle_300_6 - slope_angle_600_3,slope_signal_300_9,10min_slope_lin_reg_signal_300_9 - slope_lin_reg_signal_900_9,slope_angle_300_3 - slope_angle_900_3
Date,,,,,,,,,,,,,,,,,,,,,
2025-07-26 20:30:00,0,0.378237,1.588212,-0.977742,1.059255,0.261028,1.102774,-0.982505,-0.987571,-0.000604,...,-0.006334,-0.008887,-0.920648,0.019566,-0.935725,0.009094,-0.008663,1.069946,0.0,-0.577616
2025-07-26 21:15:00,0,0.378237,1.588212,-0.977742,1.059255,0.261028,1.102774,-1.136307,-0.987571,-0.000604,...,-0.006334,0.017918,1.091702,-0.130668,1.073741,0.009094,-0.008663,0.987571,0.0,0.059524
2025-07-26 23:05:00,1,0.378237,1.588212,-0.977742,1.059255,0.261028,1.102774,-0.655377,-0.987571,-0.000604,...,-0.006334,0.008887,0.987571,-0.032381,0.988749,0.009094,-0.008663,-1.059255,0.0,2.504577


In [142]:
### Labels

lab = pd.read_csv(root_data + 'Results/'+symbol+'_'+strategy+'_'+time_frame+'_Strategy_Gen_Labels.csv', index_col=0)
lab['Date'] = pd.to_datetime(lab['Date'])
lab.set_index('Date', inplace=True)

print(lab.columns,'\n')
print(lab.shape)
#lab.loc[lab['Open_Trade']== 1,'Type']

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR', 'kal_1',
       'kal_2', 'kal_3', 'kal_4', 'Open_Trade', 'Close_Trade', 'Entry_Date',
       'Type', 'Trade_Number', 'st_Exit_Date', 'trade type', 'st_Duration',
       'st_row_PnL_close', 'st_row_PnL_high', 'st_row_PnL_Low',
       'st_row_PnL_low', 'st_Max', 'st_Min', 'st_PnL', 'st_atr_PnL',
       'st_atr_max_PnL'],
      dtype='object') 

(267395, 28)


In [143]:
# --- Parámetros / campos
result_field = 'st_atr_max_PnL'   # métrica a evaluar

# --- Filtro de filas válidas
valid = (
    (lab['Type'] == direction) &
    (lab['Open_Trade'].isin([1, -1])) &
    (lab[result_field].notna()))


lab['label'] = np.nan
lab.loc[valid & (lab[result_field] <= 1), 'label'] = 0
lab.loc[valid & (lab[result_field] >= 1), 'label'] = 1

# --- Mantener solo filas válidas y con label
lab = lab.loc[valid & lab['label'].notna()].copy()
lab['label'] = lab['label'].astype('int8')

print('\nLabel counts = ')
print(lab['label'].value_counts(dropna=False).sort_index())



Label counts = 
label
0     9537
1    10054
Name: count, dtype: int64


In [144]:
### Merge

feat_obj = lab.merge(features_5m, left_index=True, right_index=True, how='left')
#feat_obj = feat_obj.merge(features_10m, left_index=True, right_index=True, how='left')
feat_obj.fillna(method='ffill', inplace=True)
#feat_obj.dropna(inplace=True)
print('Shape = ',feat_obj.shape,'\n')
print(list(feat_obj.columns))

feat_obj.dropna(inplace=True)
#feat_obj.tail(3)

Shape =  (19591, 301) 

['Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR', 'kal_1', 'kal_2', 'kal_3', 'kal_4', 'Open_Trade', 'Close_Trade', 'Entry_Date', 'Type', 'Trade_Number', 'st_Exit_Date', 'trade type', 'st_Duration', 'st_row_PnL_close', 'st_row_PnL_high', 'st_row_PnL_Low', 'st_row_PnL_low', 'st_Max', 'st_Min', 'st_PnL', 'st_atr_PnL', 'st_atr_max_PnL', 'label_x', 'label_y', '10min_RSI_3_diff', '10min_RSI_3', '10min_slope_signal_600_3', '10min_slope_signal_300_3', '10min_slope_div_300_3_diff', '10min_slope_lin_reg_signal_900_6', 'RSI_7', '10min_slope_signal_900_3', '10min_slope_angle_900_3_diff', '10min_slope_signal_900_9_diff', '10min_slope_signal_600_3_diff', '10min_slope_lin_reg_signal_600_3', '10min_slope_div_300_6 - slope_div_600_9', '10min_slope_div_300_9', '10min_slope_signal_300_6 - slope_signal_600_3', '10min_slope_signal_300_6', '10min_slope_signal_300_6 - slope_signal_600_6', '10min_slope_angle_600_3', '10min_slope_angle_600_3_diff', '10min_slope_lin_reg_signal_

In [145]:
columns_to_drop = ['st_row_PnL_Low']
feat_obj = feat_obj.drop(columns=columns_to_drop)
feat_obj.dropna(inplace=True)

In [146]:
# Review missing values without dropping columns
nan_counts = feat_obj.isnull().sum()
print("NaN counts in feat_obj sorted by highest to lowest:")
print(nan_counts.sort_values(ascending=False))
print("Total NaN count in feat_obj:", nan_counts.sum())

NaN counts in feat_obj sorted by highest to lowest:
slope_angle_300_3 - slope_angle_900_3    0
Open                                     0
High                                     0
Low                                      0
Close                                    0
                                        ..
st_Exit_Date                             0
Trade_Number                             0
Type                                     0
Entry_Date                               0
Close_Trade                              0
Length: 300, dtype: int64
Total NaN count in feat_obj: 0


In [147]:
# Split the data into 70% train and 30% test based on index
train_size = int(0.7 * len(feat_obj))
train = feat_obj.iloc[:train_size]
test  = feat_obj.iloc[train_size:]

print("Shape of train_data:", train.shape)
print("Shape of test_data:", test.shape)

Shape of train_data: (0, 300)
Shape of test_data: (0, 300)


In [138]:
### Define Train dataframe

print('Train_Min_Date', train.index.min())
print('Train_Max_Date', train.index.max(),'\n')

print('Test_Min_Date', test.index.min())
print('Test_Max_Date', test.index.max(),'\n')

print('Train_Columns : ',train.columns, '\n')
print('Test_Columns : ',test.columns)

Train_Min_Date nan
Train_Max_Date nan 

Test_Min_Date nan
Test_Max_Date nan 

Train_Columns :  Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR', 'kal_1',
       'kal_2', 'kal_3',
       ...
       '10min_slope_angle_600_9 - slope_angle_900_6',
       'slope_signal_600_3 - slope_signal_600_6', 'slope_lin_reg_signal_600_9',
       'slope_angle_300_3 - slope_angle_900_6', 'slope_angle_900_6',
       '10min_slope_angle_300_9 - slope_angle_600_6',
       '10min_slope_angle_300_6 - slope_angle_600_3', 'slope_signal_300_9',
       '10min_slope_lin_reg_signal_300_9 - slope_lin_reg_signal_900_9',
       'slope_angle_300_3 - slope_angle_900_3'],
      dtype='object', length=300) 

Test_Columns :  Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR', 'kal_1',
       'kal_2', 'kal_3',
       ...
       '10min_slope_angle_600_9 - slope_angle_900_6',
       'slope_signal_600_3 - slope_signal_600_6', 'slope_lin_reg_signal_600_9',
       'slope_angle_300_3 - slope_angle_900

# Results


In [139]:
results(train, pnl_column='st_PnL')

AttributeError: 'float' object has no attribute 'days'

In [ ]:
results(test, pnl_column= 'st_PnL')


# ML


## Train

In [ ]:
### When using train_test_split it applies the scaler to X_train only
start_feature = train.columns.get_loc('10min_RSI_3_diff')
train_features = [col for col in train.columns[start_feature:] if col != 'label']

X = train.loc[:, train_features]
y = train['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:",  X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:",  y_test.shape)

In [ ]:
print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
### Rebalance Data frame due to the low number of class 1
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Now train your model on the resampled data
print(X_train_resampled.value_counts(),'\n')
print(y_train_resampled.value_counts())

In [ ]:
# --------------- Classifier 1: XGBoost ------------------------------------------
xgb = XGBClassifier(n_estimators=15, max_depth=3,
                    random_state=42, eval_metric='logloss')

# --------------- Classifier 2: Logistic Regression Classifier------------------------------------------
lr = LogisticRegression(random_state=42)

# --------------- Classifier 3: AdaBoost Classifier ------------------------------------------
ada = AdaBoostClassifier(n_estimators=15, random_state=42)

# --------------- Classifier 4: SVM------------------------------------------
svc = svm.SVC(kernel='rbf', probability=True, random_state=42)

# Define a list to store the different models
estimator = []
estimator.append(('LR', lr))
estimator.append(('XGB', xgb))
estimator.append(('ada', ada))
estimator.append(('SVC', svc))

In [ ]:
# --------------- Classifier 1: XGBoost ------------------------------------------
xgb = XGBClassifier(n_estimators=15, max_depth=3,
                    random_state=42, eval_metric='logloss')

# --------------- Classifier 2: Logistic Regression Classifier------------------------------------------
lr = LogisticRegression(random_state=42)

# --------------- Classifier 3: AdaBoost Classifier ------------------------------------------
ada = AdaBoostClassifier(n_estimators=15, random_state=42)

# --------------- Classifier 4: SVM------------------------------------------
svc = svm.SVC(kernel='rbf', probability=True, random_state=42)

# Define a list to store the different models
estimator = []
estimator.append(('LR', lr))
estimator.append(('XGB', xgb))
estimator.append(('ada', ada))
estimator.append(('SVC', svc))

In [ ]:
### Implement voting classifier with hard voting

ml_model = VotingClassifier(estimators=estimator, voting='soft')
ml_model.fit(X_train_resampled, y_train_resampled)
#ml_model.fit(X_train, y_train)

In [ ]:
y_true = y_test
y_pred = ml_model.predict(X_test)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

class_report = classification_report(y_true, y_pred)
print("\nClassification Report:")
print(class_report)

In [ ]:
### Save ML Model
model_path = root_data + 'Models/'+symbol+'_'+direction+'_ml_model.joblib'
joblib.dump(ml_model, model_path)
print(f"Model saved successfully at: {model_path}")

## Meta

In [ ]:
### Import ML Model
model_path = root_data+'Models/'+symbol+'_'+direction+'_ml_model.joblib'
ml_model = joblib.load(model_path)
print(f"Model loaded successfully from: {model_path}")

In [ ]:
#train.head(5)

In [ ]:
### Import Data
start_feature = train.columns.get_loc('10min_RSI_3')
train_features = [col for col in train.columns[start_feature:] if col != 'label']

# Select the feature columns from the 'train' DataFrame
X_train_features = train[train_features]

train['label_ml'] = ml_model.predict(X_train_features)

meta_features = train_features + ['label_ml']
X_meta_features = train[meta_features]

prediction_probabilities = ml_model.predict_proba(X_train_features)
train['prob_0'] = prediction_probabilities[:, 0]
train['prob_1'] = prediction_probabilities[:, 1]

#train.head()

In [ ]:
train.to_csv(root_data + 'Results/'+symbol+'Meta_Prob_M5+M10_train_l.csv')

In [ ]:
meta = train_features + ['label_ml', 'prob_1', 'prob_0']
#meta = train_features + ['prob_1']
X_meta_features = train[meta]

X = train[meta]
y = train['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

In [ ]:
print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
### Rebalance Data frame due to the low number of class 1

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
# --------------- Classifier 1: XGBoost ------------------------------------------
xgb = XGBClassifier(n_estimators=15, max_depth=3,
                    random_state=42, eval_metric='logloss')

# --------------- Classifier 2: Logistic Regression Classifier------------------------------------------
lr = LogisticRegression(random_state=42)

# --------------- Classifier 3: AdaBoost Classifier ------------------------------------------
ada = AdaBoostClassifier(n_estimators=15, random_state=42)

# --------------- Classifier 4: SVM------------------------------------------
svc = svm.SVC(kernel='rbf', probability=True, random_state=42)

# Define a list to store the different models
estimator = []
estimator.append(('LR', lr))
estimator.append(('XGB', xgb))
estimator.append(('ada', ada))
estimator.append(('SVC', svc))

In [ ]:
### Implement voting classifier with hard voting

meta_ml_model = VotingClassifier(estimators=estimator, voting='soft')
meta_ml_model.fit(X_train_resampled, y_train_resampled)

In [ ]:
y_true = y_test
y_pred = meta_ml_model.predict(X_test)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

class_report = classification_report(y_true, y_pred)
print("\nClassification Report:")
print(class_report)

In [ ]:
train['meta_results'] = meta_ml_model.predict(X_meta_features)

In [ ]:
train.to_csv(root_data + 'Results/'+symbol+'train_l_Signals & Meta_Signals.csv')

In [ ]:
### Save ML Model
model_path = root_data + 'Models/'+symbol+'_'+direction+'_Meta_ml_model.joblib'
joblib.dump(meta_ml_model, model_path)
print(f"Model saved successfully at: {model_path}")

## PnL Train

In [ ]:
train['ml_results'] = np.where(train['label_ml']==1, train['st_PnL'],0)
results(train, pnl_column='ml_results')

In [ ]:
train['meta_ml_results'] = np.where(train['meta_results']==1, train['st_PnL'],0)
results(train, pnl_column='meta_ml_results')


# Test


## Results_ML

In [ ]:
### Import ML Model

model_path = root_data+'Models/'+symbol+'_'+direction+'_ml_model.joblib'
ml_model = joblib.load(model_path)
print(f"Model loaded successfully from: {model_path}")

Model loaded successfully from: /content/drive/MyDrive/Course Folder/Forex/XAUUSD/Models/BTCUSD_Short_ml_model.joblib


In [ ]:
### Import Meta Model

model_path = root_data+'Models/'+symbol+'_'+direction+'_Meta_ml_model.joblib'
meta_ml_model = joblib.load(model_path)
print(f"Model loaded successfully from: {model_path}")

Model loaded successfully from: /content/drive/MyDrive/Course Folder/Forex/XAUUSD/Models/BTCUSD_Short_Meta_ml_model.joblib


In [ ]:

test['label_ml'] = ml_model.predict(test.loc[:,train_features])

prediction_probabilities = ml_model.predict_proba(test.loc[:,train_features])
test['prob_0'] = prediction_probabilities[:, 0]
test['prob_1'] = prediction_probabilities[:, 1]

print(test.columns)

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR', 'kal_1',
       'kal_2', 'kal_3',
       ...
       'EMA_Crossover_7_20', 'ADX_17', 'ADX_20', 'skew_4',
       'slope_signal_500_18', 'ADX_7', 'slope_lin_reg_signal_700_6',
       'label_ml', 'prob_0', 'prob_1'],
      dtype='object', length=389)


In [ ]:
print(test.columns)

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR', 'kal_1',
       'kal_2', 'kal_3',
       ...
       'EMA_Crossover_7_20', 'ADX_17', 'ADX_20', 'skew_4',
       'slope_signal_500_18', 'ADX_7', 'slope_lin_reg_signal_700_6',
       'label_ml', 'prob_0', 'prob_1'],
      dtype='object', length=389)


In [ ]:
# Calculate and display Confusion Matrix
conf_matrix = confusion_matrix(test['label'], test['label_ml'])
print("Confusion Matrix:")
print(conf_matrix)

# Calculate and display Classification Report
class_report = classification_report(test['label'], test['label_ml'])
print("\nClassification Report:")
print(class_report)

Confusion Matrix:
[[1015 1051]
 [ 626 1618]]

Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.49      0.55      2066
           1       0.61      0.72      0.66      2244

    accuracy                           0.61      4310
   macro avg       0.61      0.61      0.60      4310
weighted avg       0.61      0.61      0.61      4310



In [ ]:
test['meta_label'] = meta_ml_model.predict(test.loc[:,meta])

prediction_probabilities = meta_ml_model.predict_proba(test.loc[:,meta])
test['meta_prob_0'] = prediction_probabilities[:, 0]
test['meta_prob_1'] = prediction_probabilities[:, 1]

print(test.columns)

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR', 'kal_1',
       'kal_2', 'kal_3',
       ...
       'skew_4', 'slope_signal_500_18', 'ADX_7', 'slope_lin_reg_signal_700_6',
       'label_ml', 'prob_0', 'prob_1', 'meta_label', 'meta_prob_0',
       'meta_prob_1'],
      dtype='object', length=392)


In [ ]:
# Calculate and display Confusion Matrix
conf_matrix = confusion_matrix(test['label'], test['meta_label'])
print("Confusion Matrix:")
print(conf_matrix)

# Calculate and display Classification Report
class_report = classification_report(test['label'], test['meta_label'])
print("\nClassification Report:")
print(class_report)

Confusion Matrix:
[[1003 1063]
 [ 628 1616]]

Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.49      0.54      2066
           1       0.60      0.72      0.66      2244

    accuracy                           0.61      4310
   macro avg       0.61      0.60      0.60      4310
weighted avg       0.61      0.61      0.60      4310



## Results_PnL

In [ ]:
test['ml_results'] = np.where(test['label_ml']==1, test['st_PnL'],0)
results(test, pnl_column='ml_results')

,Results
days,212
total_trades,"4,310"
,
income,"$318,062.91"
losses,"$-115,050.91"
profits,"$203,012.00"
,
profit_trades,"1,244"
loss_trades,"1,284"
,


In [ ]:
train['meta_ml_results'] = np.where(train['meta_results']==1, train['st_PnL'],0)
results(train, pnl_column='meta_ml_results')

,Results
days,481
total_trades,"10,054"
,
income,"$432,682.19"
losses,"$-133,910.21"
profits,"$298,771.98"
,
profit_trades,"2,569"
loss_trades,"2,358"
,


# Pendientes
